<h1>HIV-Drug Scrape</h1>
Drug reviews scraped from [Everyday Health] (https://www.everydayhealth.com) and [Drugs.com] (https://www.drugs.com) for HIV and PrEP drugs. Then vadar sentiment on reviews and analysis of the side effects.And uploaded to AWS server.

In [1]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import numpy as np
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
import time
import re  #regular expression (from bs documentation)
import selenium
from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float #Varchar?
Base = declarative_base()
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
# from config import remote_db_endpoint, remote_db_port
# from config import remote_hiv_dbname, remote_hiv_dbuser, remote_hiv_dbpwd
# from config import local_hiv_dbname, local_hiv_dbuser, local_hiv_dbpwd

Create engine to pass in Cloud MySQL Database Connection on AWS

In [ ]:
#Cloud MySQL Database Connection on AWS
engine = create_engine(f'mysql://{remote_hiv_dbuser}:{remote_hiv_dbpwd}@{remote_db_endpoint}:{remote_db_port}/{remote_hiv_dbname}')

In [ ]:
#Create a remote database engine connection
conn = engine.connect()

In [2]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

#Initiate browser w/ Splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

/usr/local/bin/chromedriver


<h3>Drug Reviews Scraped from Everyday Health</h3>

<h4>List of HIV and PrEP drugs to scrape</h4>
Note: For everydayhealth.com website searching for brand or generic name gives the same results

In [3]:
#Drug names to keep in lists
edh_all_hiv_drugs = ['Combivir', 'Emtriva', 'Epivir', 'Epizicom','Hivid','Trizivir',
    'Truvada','Videx','Viread','Zerit','Ziagen','Edurant','Intelence',
    'Rescriptor','Pifeltro','Sustiva','Viramune','Agenerase','Aptivus',
    'Crixivan','Invirase','Kaletra','Lexiva','Norvir','Prezista','Reyataz',
    'Viracept','Selzentry','Isentress','Tivicay','Vitekta','Atripla','Biktarvy',
    'Cimduo','Complera','Delstrigo','Dovato','Genvoya','Evotaz','Juluca',
    'Odefsey','Prezcobix','Symfi','Stribild','Symtuza','Triumeq','Descovy']

<h4>Scrape all drug reviews from everydayhealth.com </h4>
Note: some drugs do not have reviews, but heroku app will set up to scrape every so often to pull in new reviews.

In [9]:
#Empty lists to hold our scraped information
e_brands = []
e_what_fors = []
e_reviews = []
e_star_ratings =[]
e_dates_posted = []
e_no_reviews_drugs = []

for d in edh_all_hiv_drugs:
    # URL of page to be scraped
    url = f'https://www.everydayhealth.com/drugs/{d}/reviews'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser' 
    soup = bs(html, 'html.parser')  
    
        #Retrieve all elements that contain organization information in an iterable list
    if (soup.body.find(class_="error-404")):
        #Append to list to keep track of drugs without reviews
        e_no_reviews_drugs.append(d)
        print(f'{d}: Drug Not Found')
        
    elif (soup.body.find(class_="no-reviews")):
        #Append to list to keep track of drugs without reviews
        e_no_reviews_drugs.append(d)
        print(f'{d}: No Reviews')
    
    else:
        #First find how many pages to loop through
        html = browser.html
        soup = bs(html, 'html.parser')  
        #Finds the number of pages
        try:
            page_results = soup.body.find('div', class_='review-pagination') 
            pagation = page_results.text  #had to select this way, tried to go directly to the section however, it fails within the script
            pagation_split = pages.split()
            num_pages = int(pagation_split[2])
            print(num_pages)
            # Create variable for range
            i = num_pages + 1
        except: 
            i = 2
        #Loop through each page of reviews
        for x in range(1,i):
            results = soup.body.find_all(class_='review-container row')

            for r in results:
                #Scrape our info
                brand = d  #r.h3.span.text this would pull in generic name and some reviews dont include this
                try:
                    what = r.h3.text     #scrapes the reason why taking still need to strip other info
                except:
                    what = "HIV"
                review = r.p.text    #scrapes the review
                rating = r.find('div', class_='star-rating-print').text # Gives stars-rating with "Stars" after if want without "Stars" use .strip("Stars") after
                date = r.find('span', class_='time')['content']   #scrapes date and time posted (can split if needed)
                
                #Append scraped info to their respective lists
                e_brands.append(brand)
                e_what_fors.append(what)
                e_reviews.append(review)
                e_star_ratings.append(rating)
                e_dates_posted.append(date)
        try:
            browser.click_link_by_text('Next')
            time.sleep(10)
        except:
            print(f"{d}: Scrape Finished")
    time.sleep(20)
# print(brands, what_fors, reviews, star_ratings,dates_posted)

Combivir: No Reviews
Emtriva: No Reviews
Epivir: No Reviews
Epizicom: Drug Not Found
Hivid: Drug Not Found
Trizivir: No Reviews
Truvada: Scrape Finished
Videx: No Reviews
Viread: Scrape Finished
Zerit: No Reviews
Ziagen: No Reviews
Edurant: No Reviews
Intelence: No Reviews
Rescriptor: No Reviews
Pifeltro: No Reviews
Sustiva: No Reviews
Viramune: No Reviews
Agenerase: Drug Not Found
Aptivus: No Reviews
Crixivan: No Reviews
Invirase: No Reviews
Kaletra: No Reviews
Lexiva: No Reviews
Norvir: No Reviews
Prezista: No Reviews
Reyataz: No Reviews
Viracept: No Reviews
Selzentry: No Reviews
Isentress: Scrape Finished
Tivicay: Scrape Finished
Vitekta: No Reviews
Atripla: Scrape Finished
Biktarvy: Scrape Finished
Cimduo: No Reviews
Complera: No Reviews
Delstrigo: Scrape Finished
Dovato: Drug Not Found
Genvoya: Scrape Finished
Evotaz: No Reviews
Juluca: No Reviews
Odefsey: Drug Not Found
Prezcobix: No Reviews
Symfi: No Reviews
Stribild: No Reviews
Symtuza: Scrape Finished
Triumeq: Scrape Finished


In [89]:
#Place results into a data frame
edh_scrape_df = pd.DataFrame({'Drugs':e_brands,'Reasons':e_what_fors,'Reviews':e_reviews,
                               'Date Posted':e_dates_posted,'Ratings':e_star_ratings,'Source':'Everyday Health'})
edh_scrape_df

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...","April 11, 2018",1.0,Everyday Health
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...","August 26, 2016",10.0,Everyday Health
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...","October 18, 2015",3.0,Everyday Health
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...","August 2, 2015",10.0,Everyday Health
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...","February 2, 2015",10.0,Everyday Health
5,Truvada Rating Summary,HIV,"""I started Truvada once a day along with Isent...","August 29, 2013",10.0,Everyday Health
6,Truvada Rating Summary,HIV,"""I have been on the combination of Truvada/Pre...","December 24, 2011",9.0,Everyday Health
7,Truvada Rating Summary,HIV,"""I started use Truvada, when Intelence affecte...","September 30, 2011",10.0,Everyday Health
8,Truvada Rating Summary,HIV,"""My viral load has gone for 180,000 to undetec...","May 19, 2011",10.0,Everyday Health
9,Truvada Rating Summary,HIV,"""After one year on Truvada in combination with...","April 19, 2010",9.0,Everyday Health


In [26]:
#Strip down reason for taking to HIV treatment and PrEP (add if/else to remove any reviews for something else)
edh_scrape_df.loc[edh_scrape_df['Reasons'].str.contains('Pre|PrEP|PREP|prep'), 'Reasons']= 'PrEP' #removed prophalaxis because it interfers with how the drugs get classified
edh_scrape_df.loc[edh_scrape_df['Reasons'].str.contains('HIV'), 'Reasons'] = 'HIV'
edh_scrape_df['Ratings'] = edh_scrape_df['Ratings'].str.replace(r'Stars','')
edh_scrape_df['Ratings'] = edh_scrape_df['Ratings'].apply(lambda x: float(x))
edh_scrape_df['Ratings'] = edh_scrape_df['Ratings'].apply(lambda x: float(x))
edh_scrape_df['Date Posted'] = edh_scrape_df['Date Posted'].apply(lambda x: pd.to_datetime(x))
edh_scrape_df['Date Posted'] = edh_scrape_df['Date Posted'].apply(lambda x: pd.Timestamp.date(x))
# edh_scrape_df['Date Posted'] = edh_scrape_df['Date Posted'].apply(lambda x: str(x))

edh_scrape_df.head()

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...",2018-04-11,1.0,Everyday Health
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...",2016-08-26,10.0,Everyday Health
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...",2015-10-18,3.0,Everyday Health
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...",2015-08-02,10.0,Everyday Health
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...",2015-02-02,10.0,Everyday Health


In [27]:
# edh_scrape_df.to_csv('edh_scrape.csv', index=False)

In [90]:
edh_scrape_df = pd.read_csv('edh_scrape.csv')
edh_scrape_df

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...",2018-04-11,1.0,Everyday Health
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...",2016-08-26,10.0,Everyday Health
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...",2015-10-18,3.0,Everyday Health
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...",2015-08-02,10.0,Everyday Health
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...",2015-02-02,10.0,Everyday Health
5,Truvada Rating Summary,HIV,"""I started Truvada once a day along with Isent...",2013-08-29,10.0,Everyday Health
6,Truvada Rating Summary,HIV,"""I have been on the combination of Truvada/Pre...",2011-12-24,9.0,Everyday Health
7,Truvada Rating Summary,HIV,"""I started use Truvada, when Intelence affecte...",2011-09-30,10.0,Everyday Health
8,Truvada Rating Summary,HIV,"""My viral load has gone for 180,000 to undetec...",2011-05-19,10.0,Everyday Health
9,Truvada Rating Summary,HIV,"""After one year on Truvada in combination with...",2010-04-19,9.0,Everyday Health


<h3>Drug Reviews Scraped from Drugs.com</h3>

<h4>Lists of HIV and PrEP drugs to scrape</h4>
Note: For drugs.com website search is by specific bits of address.

In [14]:
drugs_all_hiv_drugs = ['emtricitabine-tenofovir/truvada-',
                       'efavirenz-emtricitabine-tenofovir/atripla-',
                       'ritonavir/norvir-',
                       'raltegravir/isentress-',
                       'darunavir/prezista-',
                       'cobicistat-elvitegravir-emtricitabine-tenofovir/stribild-',
                       'abacavir-dolutegravir-lamivudine/triumeq-',
                       'dolutegravir/tivicay-',
                       'emtricitabine-rilpivirine-tenofovir/complera-',
                       'abacavir-lamivudine/epzicom-',
                       'cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/genvoya-',
                       'atazanavir/reyataz-',
                       'emtricitabine-tenofovir-alafenamide/descovy-',
                       'lamivudine/',
                       'tenofovir/viread-',
                       'lamivudine/epivir-',
                       'lopinavir-ritonavir/kaletra-',
                       'efavirenz/sustiva-',
                       'efavirenz-emtricitabine-tenofovir/',
                       'etravirine/intelence-',
                       'ritonavir/',
                       'emtricitabine-tenofovir/',
                       'tenofovir/',
                       'abacavir/',
                       'bictegravir-emtricitabine-tenofovir-alafenamide/biktarvy-',
                       'abacavir-lamivudine/','efavirenz/',
                       'raltegravir/',
                       'darunavir/',
                       'dolutegravir/',
                       'abacavir/ziagen-',
                       'nevirapine/viramune-',
                       'atazanavir/',
                       'lamivudine-zidovudine/combivir-',
                       'cobicistat-darunavir/prezcobix-',
                       'emtricitabine/emtriva-',
                       'fosamprenavir/lexiva-',
                       'emtricitabine-rilpivirine-tenofovir-alafenamide/odefsey-',
                       'zidovudine/',
                       'nevirapine/',
                       'abacavir-lamivudine-zidovudine/trizivir-',
                       'indinavir/crixivan-',
                       'emtricitabine/',
                       'lopinavir-ritonavir/',
                       'stavudine/zerit-',
                       'abacavir-dolutegravir-lamivudine/',
                       'didanosine/',
                       'emtricitabine-rilpivirine-tenofovir/',
                       'etravirine/',
                       'indinavir/',
                       'lamivudine-zidovudine/',
                       'maraviroc/',
                       'stavudine/',
                       'didanosine/videx-',
                       'nelfinavir/viracept-',
                       'abacavir-lamivudine-zidovudine/',
                       'tipranavir/aptivus-',
                       'cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/',
                       'atazanavir-cobicistat/evotaz-',
                       'raltegravir/isentress-hd-',
                       'dolutegravir-rilpivirine/juluca-',
                       'bictegravir-emtricitabine-tenofovir-alafenamide/',
                       'cobicistat/',
                       'delavirdine/',
                       'doravirine/',
                       'efavirenz-lamivudine-tenofovir/',
                       'emtricitabine-rilpivirine-tenofovir-alafenamide/',
                       'emtricitabine-tenofovir-alafenamide/',
                       'enfuvirtide/',
                       'rilpivirine/',
                       'cobicistat-darunavir-emtricitabine-tenofovir-alafenamide/symtuza-',
                       'cobicistat-darunavir-emtricitabine-tenofovir-alafenamide/',
                       'dolutegravir-rilpivirine/']

drugs_prep_hiv_drugs = ['emtricitabine-tenofovir/truvada-','emtricitabine-tenofovir-alafenamide/descovy-']

In [15]:
brands = []
what_fors = []
reviews = []
dates_posted = []
star_ratings = []

for d in drugs_all_hiv_drugs:
    # URL of page to be scraped (so too many to type with, without brand and generic)
    url = f'https://www.drugs.com/comments/{d}for-hiv-infection.html'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    try:
        paging = soup.find('ul', class_='ddc-paging')
        paging_lis = paging.find_all('li')
        i = (len(paging_lis))
    except:
        i = 2


    for x in range(1,i):
        #test prints
        html = browser.html
        results = soup.body.find('div', class_='contentBox')
        branded = results.h2.text #why is this stripping the last letter?
        whats = "HIV"

        results2 = soup.body.find_all('div', class_='ddc-comment')
        for r in results2:
            #Get the review text
            review = r.p.text 
            
            #Get the date published
            try:
                date = r.find('span',class_='comment-date').text
            except:
                date = 'n/a'
            
            #Get the rating divide by 2 because it is a rating of 1-10 and we only want a rating 1-5
            try:
                rate = r.find('div', class_='rating-score').text #will have to divide by 2
                rating = int(float(rate))
            except:
                rating = np.nan  
            
            #Append scraped info to respective lists
            brands.append(branded)
            what_fors.append(whats)
            reviews.append(review)
            dates_posted.append(date)
            star_ratings.append(rating)
            
    print(f'{d}: Scrape Finished')
    time.sleep(10)
# print(brands,what_fors,reviews,dates_posted,star_ratings)

emtricitabine-tenofovir/truvada-: Scrape Finished
efavirenz-emtricitabine-tenofovir/atripla-: Scrape Finished
ritonavir/norvir-: Scrape Finished
raltegravir/isentress-: Scrape Finished
darunavir/prezista-: Scrape Finished
cobicistat-elvitegravir-emtricitabine-tenofovir/stribild-: Scrape Finished
abacavir-dolutegravir-lamivudine/triumeq-: Scrape Finished
dolutegravir/tivicay-: Scrape Finished
emtricitabine-rilpivirine-tenofovir/complera-: Scrape Finished
abacavir-lamivudine/epzicom-: Scrape Finished
cobicistat-elvitegravir-emtricitabine-tenofovir-alafenamide/genvoya-: Scrape Finished
atazanavir/reyataz-: Scrape Finished
emtricitabine-tenofovir-alafenamide/descovy-: Scrape Finished
lamivudine/: Scrape Finished
tenofovir/viread-: Scrape Finished
lamivudine/epivir-: Scrape Finished
lopinavir-ritonavir/kaletra-: Scrape Finished
efavirenz/sustiva-: Scrape Finished
efavirenz-emtricitabine-tenofovir/: Scrape Finished
etravirine/intelence-: Scrape Finished
ritonavir/: Scrape Finished
emtricitab

In [16]:
#Scrape PrEP drugs
for d in drugs_prep_hiv_drugs:
    # URL of page to be scraped (so too many to type with, without brand and generic)
    url = f'https://www.drugs.com/comments/{d}for-pre-exposure-prophylaxis.html'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser'
    soup = bs(html, 'html.parser')
    #Retrieve all elements that contain organization information in an iterable list
    try:
        paging = soup.find('ul', class_='ddc-paging')
        paging_lis = paging.find_all('li')
        i = (len(paging_lis))
    except:
        i = 2

    for x in range(1,i):
        #test prints
        html = browser.html
        results = soup.body.find('div', class_='contentBox')
        branded = results.h2.text #why is this stripping the last letter?
        whats = "PrEP"

        results2 = soup.body.find_all('div', class_='ddc-comment')
        for r in results2:
            #Get the review text
            review = r.p.text 
            
            #Get the date published
            try:
                date = r.find('span',class_='comment-date').text
            except:
                date = 'n/a'
            
            #Get the rating divide by 2 because it is a rating of 1-10 and we only want a rating 1-5
            try:
                rate = r.find('div', class_='rating-score').text #will have to divide by 2
                rating = (float(rate)/2)
            except:
                rating = np.nan  
            
            #Append scraped info to respective lists
            brands.append(branded)
            what_fors.append(whats)
            reviews.append(review)
            dates_posted.append(date)
            star_ratings.append(rating)
    
    print(f'{d}: Scrape Finished')
    time.sleep(10)
# print(brands,what_fors,reviews,dates_posted,star_ratings)

emtricitabine-tenofovir/truvada-: Scrape Finished
emtricitabine-tenofovir-alafenamide/descovy-: Scrape Finished


In [17]:
drugs_scrape_df = pd.DataFrame({'Drugs':brands,'Reasons':what_fors,'Reviews':reviews,
                               'Date Posted':dates_posted,'Ratings':star_ratings,'Source':"drugs.com"})
drugs_scrape_df.head()

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...","April 11, 2018",1.0,drugs.com
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...","August 26, 2016",10.0,drugs.com
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...","October 18, 2015",3.0,drugs.com
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...","August 2, 2015",10.0,drugs.com
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...","February 2, 2015",10.0,drugs.com


In [83]:
#Clean up scraped data
drugs_scrape_df['Drugs'] = drugs_scrape_df['Drugs'].str.replace(r'Rating Summary','')
drugs_scrape_df['Drugs'] = drugs_scrape_df['Drugs'].str.replace(r'Reviews for','')
drugs_scrape_df['Drugs'] = drugs_scrape_df['Drugs'].str.strip()
drugs_scrape_df['Reviews'] = drugs_scrape_df['Reviews'].str.replace(r'"','')
drugs_scrape_df['Date Posted'] = drugs_scrape_df['Date Posted'].apply(lambda x: pd.to_datetime(x,))
drugs_scrape_df['Ratings'] = drugs_scrape_df['Ratings'].apply(lambda x: float(x))
# drugs_scrape_df['Ratings'] = drugs_scrape_df['Ratings'].apply(lambda x: x/2)
drugs_scrape_df.head()

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada,HIV,Doctor put me truvada as part of my medicatio...,2018-04-11,1.0,drugs.com
1,Truvada,HIV,No side effects experienced in almost two year...,2016-08-26,10.0,drugs.com
2,Truvada,HIV,I`ve been on truvada daily for more than 5 yea...,2015-10-18,3.0,drugs.com
3,Truvada,HIV,I have been taking Truvuda and Edurant combin...,2015-08-02,10.0,drugs.com
4,Truvada,HIV,Been on Truvada/Norvir/Prezista combination fo...,2015-02-02,10.0,drugs.com


In [84]:
#First found all unique values in table
#drugs_scrape_df.Drugs.unique()

#Dictionary with unique values to replace with brand drug names
drug_replacements_dict = {'Cobicistat/elvitegravir/emtricitabine/tenofovir alafenamide':'Genvoya',
                          'Abacavir/dolutegravir/lamivudine':'Triumeq',
                          'Efavirenz/emtricitabine/tenofovir':'Atripla',
                          'Bictegravir/emtricitabine/tenofovir alafenamide':'Biktarvy',
                          'Emtricitabine/rilpivirine/tenofovir':'Complera',
                          'Dolutegravir':'Tivicay','Raltegravir':'Isentress',
                          'Emtricitabine/tenofovir':'Truvada','Efavirenz':'Sustiva',
                          'Abacavir/lamivudine':'Epzicom','Emtricitabine/rilpivirine/tenofovir alafenamide':'Odefsey',
                          'Darunavir':'Prezista','Nevirapine':'Viramune','Emtricitabine/tenofovir alafenamide':'Descovy',
                          'Lopinavir/ritonavir':'Kaletra','Enfuvirtide':'Fuzeon',
                          'Dolutegravir/rilpivirine':'Juluca','Maraviroc':'Selzentry',
                          'Lamivudine/zidovudine':'Combivir','Etravirine':'Intelence',
                          'Lamivudine':'Epivir','Rilpivirine':'Edurant','Ritonavir':'Norvir',
                          'Abacavir/lamivudine/zidovudine':'Trizivir','Atazanavir':'Reyataz',
                          'Abacavir':'Ziagen','Cobicistat/darunavir/emtricitabine/tenofovir alafenamide':'Symtuza'
                         }

drugs_scrape_df['Drugs'] = drugs_scrape_df['Drugs'].map(drug_replacements_dict).fillna(drugs_scrape_df['Drugs'])
drugs_scrape_df.head()

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada,HIV,Doctor put me truvada as part of my medicatio...,2018-04-11,1.0,drugs.com
1,Truvada,HIV,No side effects experienced in almost two year...,2016-08-26,10.0,drugs.com
2,Truvada,HIV,I`ve been on truvada daily for more than 5 yea...,2015-10-18,3.0,drugs.com
3,Truvada,HIV,I have been taking Truvuda and Edurant combin...,2015-08-02,10.0,drugs.com
4,Truvada,HIV,Been on Truvada/Norvir/Prezista combination fo...,2015-02-02,10.0,drugs.com


In [85]:
#Check to make sure all have changed
drugs_scrape_df.Drugs.unique()

array(['Truvada', 'Atripla', 'Norvir', 'Isentress', 'Prezista',
       'Stribild', 'Triumeq', 'Tivicay', 'Complera', 'Epzicom', 'Genvoya',
       'Reyataz', 'Descovy', 'Epivir', 'Kaletra', 'Sustiva', 'Intelence',
       'Ziagen', 'Biktarvy', 'Viramune', 'Combivir', 'Prezcobix',
       'Odefsey', 'Trizivir', 'Selzentry', 'Juluca', 'Fuzeon', 'Edurant',
       'Symtuza'], dtype=object)

In [86]:
drugs_scrape_df.to_csv('drugs_scrape.csv', index=False)

Combine both data frames

In [87]:
drug_reviews_df = edh_scrape_df.append(drugs_scrape_df)
drug_reviews_df.head()

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...",2018-04-11,1.0,Everyday Health
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...",2016-08-26,10.0,Everyday Health
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...",2015-10-18,3.0,Everyday Health
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...",2015-08-02,10.0,Everyday Health
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...",2015-02-02,10.0,Everyday Health


In [29]:
#Drug classes and abbrv's NRTs:Nucleoside Reverse Transcriptase Inhibitor; NNRTIs: Non-Nucleoside
#Reverse Transcriptase Inhibitor; PIs: Protease Inhibitor; CAs:CCR5 Antagonist; INSTIs: Integrase
# Inhibitor; MCCs: Multi-Class Combination; PrEPs: Pre-Exposure Prophylaxis; PEPs: Post-Exposure Prophylaxis
hiv_drug_classes = ['NRTs','NNRTIs','PIs','CAs','INSTIs','MCCs','PrEPs','PEPs']
nrts = ['Combivir', 'Descovy', 'Emtriva', 'Epivir', 'Epizicom','Hivid','Trizivir',
            'Videx','Viread','Zerit','Ziagen','Truvada']
nnrtis = ['Edurant','Intelence','Rescriptor','Pifeltro','Sustiva','Viramune']
pis = ['Agenerase','Aptivus','Crixivan','Invirase','Kaletra','Lexiva','Norvir',
      'Prezista','Reyataz','Viracept']
cas = ['Selzentry']
instis = ['Isentress','Tivicay','Vitekta']
mccs = ['Atripla','Biktarvy','Cimduo','Complera','Delstrigo','Dovato','Genvoya',
       'Evotaz','Juluca','Odefsey','Prezcobix','Symfi','Stribild','Symtuza','Triumeq']
preps = ['Truvada','Descovy']

In [82]:
#Classify each drug
# classify = []
# for drug in drug_reviews_df['Drugs']:
#     if drug.isin([mccs]):
#         classify.append('MMC')
#     elif drug.isin([instis]):
#         classify.append('INSTI')
#     elif drug.isin([nnrtis]):
#         classify.append('NNRTI')
#     elif drug.isin([nrts]):
#         classify.append('NRT')
#     elif drug.isin([pis]):
#         classify.append('PI')
#     else:
#         classify.append('None')
# classify
    
drug_reviews_df['Classify'] = pd.np.where(drug_reviews_df.Drugs.isin(mccs), 'MMC',
                    pd.np.where(drug_reviews_df.Drugs.isin(instis), 'INSTI',
                    pd.np.where(drug_reviews_df.Drugs.isin(nnrtis), 'NNRTI',
                    pd.np.where(drug_reviews_df.Drugs.isin(nrts), 'NRT',
                    pd.np.where(drug_reviews_df.Drugs.isin(pis), 'PI','None'            
                    )))))

drug_reviews_df.loc[drug_reviews_df['Reasons'].str.contains('PrEP'), 'Classify'] = 'PrEP'
drug_reviews_df

,Drugs,Reasons,Reviews,Date Posted,Ratings,Source,Classify
0,Truvada Rating Summary,HIV,"""Doctor put me truvada as part of my medicati...",2018-04-11,1.0,Everyday Health,None
1,Truvada Rating Summary,HIV,"""No side effects experienced in almost two yea...",2016-08-26,10.0,Everyday Health,None
2,Truvada Rating Summary,HIV,"""I`ve been on truvada daily for more than 5 ye...",2015-10-18,3.0,Everyday Health,None
3,Truvada Rating Summary,HIV,"""I have been taking Truvuda and Edurant combi...",2015-08-02,10.0,Everyday Health,None
4,Truvada Rating Summary,HIV,"""Been on Truvada/Norvir/Prezista combination f...",2015-02-02,10.0,Everyday Health,None
5,Truvada Rating Summary,HIV,"""I started Truvada once a day along with Isent...",2013-08-29,10.0,Everyday Health,None
6,Truvada Rating Summary,HIV,"""I have been on the combination of Truvada/Pre...",2011-12-24,9.0,Everyday Health,None
7,Truvada Rating Summary,HIV,"""I started use Truvada, when Intelence affecte...",2011-09-30,10.0,Everyday Health,None
8,Truvada Rating Summary,HIV,"""My viral load has gone for 180,000 to undetec...",2011-05-19,10.0,Everyday Health,None
9,Truvada Rating Summary,HIV,"""After one year on Truvada in combination with...",2010-04-19,9.0,Everyday Health,None


In [ ]:
# Initialize Sentiment Analyzer
analyzer = SentimentIntensityAnalyzer()

In [ ]:
# Variables for holding sentiments
compound_list = []
positive_list = []
neutral_list = []
negative_list = []        
score = []

vader_review_analysis = drugs_scrape_df
# Loop through all tweets
for review in vader_review_analysis['Reviews']:

    # Run Vader Analysis on each tweet
    results = analyzer.polarity_scores(review)
    compound = results["compound"]
    pos = results["pos"]
    neu = results["neu"]
    neg = results["neg"]

    # Add each value to the appropriate list
    compound_list.append(compound)
    positive_list.append(pos)
    negative_list.append(neg)
    neutral_list.append(neu)
    
    #Make Vader "Score" that is tangible for the public
    if compound <= (-.6):
        score.append('Very Negative')
    elif compound <= (-.2):
        score.append('Negative')
    elif compound <= .2:
        score.append('Neutral')
    elif compound <= .6:
        score.append('Positive')
    else:
        score.append('Very Positive')
    
#Add to dataframe
vader_review_analysis['Compound'] = compound_list
vader_review_analysis['Positive'] = positive_list
vader_review_analysis['Neutral'] = neutral_list
vader_review_analysis['Negative'] = negative_list
vader_review_analysis['Score'] = score
    
# # Print the Averages
# print(f"User: {target_user}")
# print(f"Compound: {np.mean(compound_list):.3f}")
# print(f"Positive: {np.mean(positive_list):.3f}")
# print(f"Neutral: {np.mean(neutral_list):.3f}")
# print(f"Negative: {np.mean(negative_list):.3f}")

# print(compound_list,positive_list,negative_list,neutral_list)
print(vader_review_analysis)

In [ ]:
#Make stats_by_drug dataframe
stats_by_drug = vader_review_analysis.groupby('Drug').mean()
stats_by_drug['Counts'] = vader_review_analysis.groupby('Drug').size()
stats_by_drug

In [ ]:
#Make stats_by_classify dataframe
stats_by_classify = vader_review_analysis.groupby('Classify').mean()
stats_by_classify['Counts'] = vader_review_analysis.groupby('Drug').size()
stats_by_classify

In [ ]:
#Make stats_reason_compare (Truvada and Descovy ONLY!) ***
class_averages_df = vader_review_analysis.groupby('Classify').mean()
class_averages_df['Counts'] = vader_review_analysis.groupby('Drug').size()#use class_averages_df?
class_averages_df

In [ ]:
#vader_counts = pd.DataFrame(vader_review_analysis['Score'].value_counts())
vader_counts = vader_review_analysis['Score'].value_counts()
vader_counts

In [ ]:
#reason_counts = pd.DataFrame(vader_review_analysis['Reason'].value_counts())
reason_counts = vader_review_analysis['Reasons'].value_counts()
reason_counts

In [ ]:
#stars_counts = pd.DataFrame(vader_review_analysis['Ratings'].value_counts())
stars_counts = vader_review_analysis['Ratings'].value_counts()
stars_counts

In [ ]:
#To work on

In [52]:
brand = []
side_effects = []
reason = []

for d in edh_all_hiv_drugs:
    # URL of page to be scraped
    url = f'https://www.rxlist.com/{d}-side-effects-drug-center.htm'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser' 
    soup = bs(html, 'html.parser')

    results = soup.body.find('div', class_='pgContent')
    results_lists = results.find_all('li')
    for r in results_lists:
        effects_text = r.text
        effects_strip = effects_text.strip()
        print(effects_strip)
        
        brand.append(d)
        side_effects.append(effects_strip)
        reason.append('HIV')

#Scrape for just prep drugs
prep_brands = ['Truvada','Descovy']
for p in prep_brands:
    # URL of page to be scraped
    url = f'https://www.rxlist.com/{p}-side-effects-drug-center.htm'
    #adding in splinter needed item
    browser.visit(url)
    html = browser.html
    # Create BeautifulSoup object; parse with 'html.parser' 
    soup = bs(html, 'html.parser')

    results = soup.body.find('div', class_='pgContent')
    results_lists = results.find_all('li')
    for r in results_lists:
        effects_text = r.text
        effects_strip = effects_text.strip()
        print(effects_strip)
        
        brand.append(p)
        side_effects.append(effects_strip)
        reason.append('PrEP')
        
        
print(brand,side_effects,reason)

headache,
dizziness,
nausea,
loss of appetite,
diarrhea,
trouble sleeping,
cold symptoms (stuffy nose, sneezing, sinus pain, cough), or
changes in the shape or location of body fat (especially in your arms, legs, face, neck, breasts, and trunk).
mental/mood changes (such as depression),
signs of pancreatitis (such as vomiting, stomach/abdominal/back pain, fever),
easy bruising or bleeding, or
skin or nail color changes.
headache
nausea
upset stomach
diarrhea
trouble sleeping
darkening skin color on palms of hands and soles of feet
dizziness
strange dreams
cough
runny nose
skin rash or changes in the shape or location of body fat (especially in the arms legs face neck breasts 
	and trunk)
unexplained weight loss,
persistent muscle aches or weakness,
joint pain,
numbness or tingling of the hands/feet/arms/legs,
severe tiredness,
vision changes,
severe or persistent headaches,
signs of infection (such as fever, chills, trouble breathing, cough, non-healing skin sores),
signs of an overact

Drugs A-Z List - by drug brand and generic 
	name.
Pill Identifier Tool 
	- identify drugs and medications by imprint, color, or shape. Includes drug 
	pictures.
Picture Slideshows - 
	interactive medical slideshow on important medical topics.
Image Gallery - 
	learn about skin disorder, medical anatomy, tumors and more through images and illustrations .
Latest Drug News - 
	WebMD & FDA Updates and news releases on prescription medications and drug 
	manufacturing/distribution.
Diseases, 
	Condition and Tests - The RxList disease and condition listing contains 
	explanations of many diseases, conditions, medical procedures and tests.
Drug Medical Dictionary 
	- definitions and explanations of many medical terms including prescription 
	medication abbreviations.
Neurontin (gabapentin)
Celexa (citalopram hydrobromide)
Xanax (CIV alprazolam)
Klonopin (clonazepam)
Cipro (ciprofloxacin hydrochloride)
Percocet (oxycodone and acetaminophen)
Vicodin ((hydrocodone bitartrate and 
		acetaminophe

yellowing of the skin or eyes (jaundice),
itching and rash,
hives,
nausea,
diarrhea,
vomiting,
abdominal pain,
fatigue,
muscle tissue breakdown (rhabdomyolysis),
headache,
depression,
abnormal or strange dreams,
insomnia, and
kidney disease.
diarrhea and
headache.
other antiretroviral medications,
dofetilide,
metformin,
antacids,
anticonvulsants,
antimycobacterials,
glucocorticoids,
H2-receptor antagonists,
St John's wort,
macrolide or ketolide antibiotics,
cation-containing products or laxatives,
sucralfate,
buffered medications,
narcotic analgesics,
oral calcium and iron supplements (including multivitamins containing calcium or iron),
and proton pump inhibitors (PPIs).
depressive disorders (depressed mood, depression, general unease, mood changes, negative thoughts, suicide attempts, suicidal ideation),
insomnia,
headache,
nausea,
drowsiness,
dizziness,
abdomincal pain,
rash,
weight gain, and
redistribution of body fat.
diarrhea,
nausea,
rash,
headache,
abdominal pain,
vomiting, and

In [53]:
brand_side_effects_df = pd.DataFrame({'Drugs':brand,'Side_Effects':side_effects,'Reasons':reason})
brand_side_effects_df.head()

,Drugs,Side_Effects,Reasons
0,Combivir,"headache,",HIV
1,Combivir,"dizziness,",HIV
2,Combivir,"nausea,",HIV
3,Combivir,"loss of appetite,",HIV
4,Combivir,"diarrhea,",HIV


In [56]:
# brand_side_effects_df['Side_Effects'].replace({",":"",".":"",";":""})
effects_clean = []
for effect in brand_side_effects_df['Side_Effects']:
    effect = effect.replace(',','')
    effect = effect.replace('.','')
    effect = effect.replace('/',' ')
    effects_clean.append(effect)
# effects_clean

brand_side_effects_df['Side_Effects'] = effects_clean
brand_side_effects_df.head()

,Drugs,Side_Effects,Reasons
0,Combivir,headache,HIV
1,Combivir,dizziness,HIV
2,Combivir,nausea,HIV
3,Combivir,loss of appetite,HIV
4,Combivir,diarrhea,HIV


In [59]:
#Classify each drug
brand_side_effects_df['Classify'] = pd.np.where(brand_side_effects_df.Drugs.isin(mccs), 'MMC',
                    pd.np.where(brand_side_effects_df.Drugs.isin(instis), 'INSTI',
                    pd.np.where(brand_side_effects_df.Drugs.isin(nnrtis), 'NNRTI',
                    pd.np.where(brand_side_effects_df.Drugs.isin(nrts), 'NRT',
                    pd.np.where(brand_side_effects_df.Drugs.isin(pis), 'PI', 'None'            
                    )))))

brand_side_effects_df.loc[brand_side_effects_df['Reasons'].str.contains('PrEP'), 'Classify'] = 'PrEP'

brand_side_effects_df.head()

,Drugs,Side_Effects,Reasons,Classify
0,Combivir,headache,HIV,NRT
1,Combivir,dizziness,HIV,NRT
2,Combivir,nausea,HIV,NRT
3,Combivir,loss of appetite,HIV,NRT
4,Combivir,diarrhea,HIV,NRT


In [ ]:
# brand_side_effects_df.to_csv('brand_side_effects.csv', index=False)

In [63]:
side_effects_only = pd.DataFrame(brand_side_effects_df['Side_Effects'])
side_effects_only.head()

,Side_Effects
0,headache
1,dizziness
2,nausea
3,loss of appetite
4,diarrhea


In [70]:
#Looking for duplicates in side effects and de-duplicate
# side_effects_only['Side_Effects'].unique()
# side_effects_only['Side_Effects'].value_counts()
side_effects_only.drop_duplicates(keep='first', inplace=True)
side_effects_only

,Side_Effects
0,headache
1,dizziness
2,nausea
3,loss of appetite
4,diarrhea
5,trouble sleeping
6,cold symptoms (stuffy nose sneezing sinus pain...
7,changes in the shape or location of body fat (...
8,mental mood changes (such as depression)
9,signs of pancreatitis (such as vomiting stomac...


In [71]:
# side_effects_only.to_csv('side_effects_only_draft.csv', index=False)

In [79]:
side_effect_types = ['Allergic', 'Physical Changes', 'Mental', 'Healt Related', 'Pain']


In [ ]:

#List of most common side effects. adding varied language and prurals to capture as many mentions as possible.
side_effects = ['weight','gain','pounds','bloat','bloated','bloating','bleed','bled','bleeding','blood','heavy','spot','spotting','headache',
                'headaches','migraine','migraines','depression','depressed','anxiety','anxious','sad','cry','cried','crying','nausea','nauseous','nauseated',
                'mood','swings','libido','drive','discharge','pain','painful','cramp','cramps','cramping','clot','clots','acne','pimples','breakout','breakouts','zits',
                'infection','vaginosis','vaginitis','bacteria','itchy','itching','dry','dryness','tired','drowsy','drowsiness','fatigue','irritable','irritablility','flashes',
                'hair','cyst','cysts','dizzy','dizziness','vomit','vomitting']


In [ ]:
# Create a funtion to consolidate different words referring to the same side effect
def clean_words(word_set):
    word_set['effects'] = word_set['effects'].replace({'bleed':'bleeding',\
                                                       'bled':'bleeding',\
                                                       'blood':'bleeding', \
                                                       'depressed':'depression', \
                                                       'anxious':'anxiety', \
                                                       'zits':'acne',\
                                                       'pimples':'acne',\
                                                       'breakout':'acne',\
                                                       'breakouts':'acne',\
                                                       'drive':'libido',\
                                                       'pain':'painful',\
                                                       'cramp':'cramps',\
                                                       'cramping':'cramps',\
                                                       'spot':'spotting',\
                                                       'bloat':'bloating',\
                                                       'bloated':'bloating',\
                                                       'cried':'crying',\
                                                       'cry':'crying',\
                                                       'nauseous':'nausea',\
                                                       'nauseated':'nausea',\
                                                       'clot':'clots',
                                                       'headache':'headaches',\
                                                       'migraine':'migraines',\
                                                       'vaginosis':'infection',\
                                                       'vaginitis':'infection',\
                                                       'bacteria':'infection'
                                                      })